In [ ]:
!git clone https://github.com/anas-rz/k3im.git

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

In [2]:
!pip install keras medmnist -qq --upgrade

/bin/bash: /home/anas/miniconda3/envs/keras_core/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Keras version: 3.0.0


In [3]:
import sys
sys.path.append('/content/k3im')

In [4]:
import keras
import medmnist
import numpy as np
import tensorflow as tf # For data processes only

2023-12-05 14:51:18.034759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
DATASET_NAME = "organmnist3d"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
num_classes = 11

def download_and_prepare_dataset(data_info: dict):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = keras.utils.get_file(origin=data_info["url"], md5_hash=data_info["MD5"])

    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )


# Get the metadata of the dataset
info = medmnist.INFO[DATASET_NAME]

# Get the dataset
prepared_dataset = download_and_prepare_dataset(info)
(train_videos, train_labels) = prepared_dataset[0]
(valid_videos, valid_labels) = prepared_dataset[1]
(test_videos, test_labels) = prepared_dataset[2]

In [6]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(train_videos, train_labels, "train")
validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
testloader = prepare_dataloader(test_videos, test_labels, "test")

2023-12-05 14:51:19.335470: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [7]:
batch_size = 32
epochs = 2
def train_model(model):
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
    model.fit(trainloader, epochs=epochs, validation_data=validloader)
    score = model.evaluate(testloader)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [8]:
from k3im.cct_3d import CCT3DModel
model = CCT3DModel(input_shape=(28, 28, 28, 1),
    num_heads=4,
    projection_dim=64,
    kernel_size=4,
    stride=4,
    padding=2,
    transformer_units=[16, 64],
    stochastic_depth_rate=0.6,
    transformer_layers=2,
    num_classes=num_classes,
    positional_emb=False,)

CUDA backend failed to initialize: Found CUDA version 11070, but JAX was built against version 11080, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [9]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28,    │       0 │ -                    │
│ (InputLayer)        │ 28, 1)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ cct_tokenizer3d     │ (None, 27, 64)    │ 266,240 │ input_layer[0][0]    │
│ (CCTTokenizer3D)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalization │ (None, 27, 64)    │     128 │ cct_tokenizer3d[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 27, 64)    │  66,368 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth    │ (None, 27, 64)    │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 27, 64)    │       0 │ stochastic_depth[0]… │
│                     │                   │         │ cct_tokenizer3d[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 27, 64)    │     128 │ add[0][0]            │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense (Dense)       │ (None, 27, 16)    │   1,040 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_1 (Dropout) │ (None, 27, 16)    │       0 │ dense[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_1 (Dense)     │ (None, 27, 64)    │   1,088 │ dropout_1[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_2 (Dropout) │ (None, 27, 64)    │       0 │ dense_1[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_1  │ (None, 27, 64)    │       0 │ dropout_2[0][0]      │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_1 (Add)         │ (None, 27, 64)    │       0 │ stochastic_depth_1[… │
│                     │                   │         │ add[0][0]            │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 27, 64)    │     128 │ add_1[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 27, 64)    │  66,368 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_2  │ (None, 27, 64)    │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_2 (Add)         │ (None, 27, 64)    │       0 │ stochastic_depth_2[… │
│                     │                   │         │ add_1[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 27, 64)    │     128 │ add_2[0][0]          │
│ (LayerNormalizatio… │                   │         │                    

 Total params: 404,652 (1.54 MB)

 Trainable params: 404,652 (1.54 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 229ms/step - accuracy: 0.1104 - loss: 2.5978 - val_accuracy: 0.0870 - val_loss: 2.2054
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.2255 - loss: 2.1498 - val_accuracy: 0.3665 - val_loss: 1.5470
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3300 - loss: 1.7090
Test loss: 1.7021818161010742
Test accuracy: 0.32786884903907776


In [11]:
from k3im.convmixer_3d import ConvMixer3DModel
model = ConvMixer3DModel(image_size=28,
    num_frames=28,
    filters=32,
    depth=2,
    kernel_size=4,
    kernel_depth=3,
    patch_size=3,
    patch_depth=3,
    num_classes=10,
    num_channels=1)

In [12]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 28, 28,    │       0 │ -                    │
│ (InputLayer)        │ 28, 1)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rescaling           │ (None, 28, 28,    │       0 │ input_layer_2[0][0]  │
│ (Rescaling)         │ 28, 1)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 9, 9, 9,   │     896 │ rescaling[0][0]      │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation          │ (None, 9, 9, 9,   │       0 │ conv3d_2[0][0]       │
│ (Activation)        │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalization │ (None, 9, 9, 9,   │     128 │ activation[0][0]     │
│ (BatchNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2_plus1d        │ (None, 9, 9, 9,   │  19,520 │ batch_normalization… │
│ (Conv2Plus1D)       │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_1        │ (None, 9, 9, 9,   │       0 │ conv2_plus1d[0][0]   │
│ (Activation)        │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 9,   │     128 │ activation_1[0][0]   │
│ (BatchNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_4 (Add)         │ (None, 9, 9, 9,   │       0 │ batch_normalization… │
│                     │ 32)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv3d_5 (Conv3D)   │ (None, 9, 9, 9,   │   1,056 │ add_4[0][0]          │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_2        │ (None, 9, 9, 9,   │       0 │ conv3d_5[0][0]       │
│ (Activation)        │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 9,   │     128 │ activation_2[0][0]   │
│ (BatchNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2_plus1d_1      │ (None, 9, 9, 9,   │  19,520 │ batch_normalization… │
│ (Conv2Plus1D)       │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_3        │ (None, 9, 9, 9,   │       0 │ conv2_plus1d_1[0][0] │
│ (Activation)        │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 9,   │     128 │ activation_3[0][0]   │
│ (BatchNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_5 (Add)         │ (None, 9, 9, 9,   │       0 │ batch_normalization… │
│                     │ 32)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv3d_8 (Conv3D)   │ (None, 9, 9, 9,   │   1,056 │ add_5[0][0]        

 Total params: 43,018 (168.04 KB)

 Trainable params: 42,698 (166.79 KB)

 Non-trainable params: 320 (1.25 KB)

In [13]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - accuracy: 0.2658 - loss: 1.7671 - val_accuracy: 0.0932 - val_loss: 2.0907
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.4928 - loss: 1.3977 - val_accuracy: 0.0932 - val_loss: 2.0937
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1219 - loss: 2.0012
Test loss: 2.020193576812744
Test accuracy: 0.10819672048091888


In [14]:
from k3im.eanet3d import EANet3DModel

In [15]:
model = EANet3DModel(
    image_size=28,
    image_patch_size=7,
    frames=28,
    frame_patch_size=7,
    num_classes=num_classes,
    dim=64,
    depth=2,
    heads=4,
    mlp_dim=32,
    channels=1,
    dim_coefficient=4,
    projection_dropout=0.0,
    attention_dropout=0,
)

In [16]:
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 28, 28,    │       0 │ -                    │
│ (InputLayer)        │ 28, 1)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape (Reshape)   │ (None, 7, 4, 7,   │       0 │ input_layer_5[0][0]  │
│                     │ 4, 7, 4, 1)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose           │ (None, 4, 4, 4,   │       0 │ reshape[0][0]        │
│ (Transpose)         │ 7, 7, 7, 1)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_1 (Reshape) │ (None, 4, 4, 4,   │       0 │ transpose[0][0]      │
│                     │ 343)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │     686 │ reshape_1[0][0]      │
│ (LayerNormalizatio… │ 343)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_7 (Dense)     │ (None, 4, 4, 4,   │  22,016 │ layer_normalization… │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │     128 │ dense_7[0][0]        │
│ (LayerNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_2 (Reshape) │ (None, 64, 64)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_8 (Dense)     │ (None, 64, 256)   │  16,640 │ reshape_2[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_3 (Reshape) │ (None, 64, 16,    │       0 │ dense_8[0][0]        │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_1         │ (None, 16, 64,    │       0 │ reshape_3[0][0]      │
│ (Transpose)         │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_9 (Dense)     │ (None, 16, 64,    │     272 │ transpose_1[0][0]    │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax_2 (Softmax) │ (None, 16, 64,    │       0 │ dense_9[0][0]        │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda (Lambda)     │ (None, 16, 64,    │       0 │ softmax_2[0][0]      │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_6 (Dropout) │ (None, 16, 64,    │       0 │ lambda[0][0]         │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_10 (Dense)    │ (None, 16, 64,    │     272 │ dropout_6[0][0]      │
│                     │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_2         │ (None, 64, 16,    │       0 │ dense_10[0][0]       │
│ (Transpose)         │ 16)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 99,577 (388.97 KB)

 Trainable params: 99,577 (388.97 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 127ms/step - accuracy: 0.3240 - loss: 2.0638 - val_accuracy: 0.9130 - val_loss: 0.5357
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7652 - loss: 0.8392 - val_accuracy: 0.9317 - val_loss: 0.2855
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7249 - loss: 0.9507
Test loss: 0.9184524416923523
Test accuracy: 0.72295081615448


In [18]:
from k3im.gmlp_3d import gMLP3DModel
model = gMLP3DModel(
    image_size=28,
    image_patch_size=7,
    frames=28,
    frame_patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=4,
    hidden_units=32,
    dropout_rate=0.4,
    channels=1,
)

In [19]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.3039 - loss: 2.1179 - val_accuracy: 0.7826 - val_loss: 0.9232
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7200 - loss: 1.0776 - val_accuracy: 0.8820 - val_loss: 0.4571
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6478 - loss: 1.1318
Test loss: 1.0548360347747803
Test accuracy: 0.6770491600036621


In [20]:
from k3im.mlp_mixer_3d import MLPMixer3DModel

model = MLPMixer3DModel(
    image_size=28,
    image_patch_size=7,
    frames=28,
    frame_patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=4,
    hidden_units=32,
    dropout_rate=0.4,
    channels=1,
)
model.summary()

Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 28, 28, 28, 1)     │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_10 (Reshape)            │ (None, 7, 4, 7, 4, 7, 4,  │          0 │
│                                 │ 1)                        │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transpose_6 (Transpose)         │ (None, 4, 4, 4, 7, 7, 7,  │          0 │
│                                 │ 1)                        │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_11 (Reshape)            │ (None, 4, 4, 4, 343)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization_20          │ (None, 4, 4, 4, 343)      │        686 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_35 (Dense)                │ (None, 4, 4, 4, 32)       │     11,008 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization_21          │ (None, 4, 4, 4, 32)       │         64 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_12 (Reshape)            │ (None, 64, 32)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ mlp_mixer_layer (MLPMixerLayer) │ (None, 64, 32)            │     12,576 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ mlp_mixer_layer_1               │ (None, 64, 32)            │     12,576 │
│ (MLPMixerLayer)                 │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ mlp_mixer_layer_2               │ (None, 64, 32)            │     12,576 │
│ (MLPMixerLayer)                 │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ mlp_mixer_layer_3               │ (None, 64, 32)            │     12,576 │
│ (MLPMixerLayer)                 │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ avg_pool                        │ (None, 32)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_52 (Dense)                │ (None, 11)                │        363 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 62,425 (243.85 KB)

 Trainable params: 62,425 (243.85 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - accuracy: 0.2796 - loss: 2.1830 - val_accuracy: 0.8820 - val_loss: 0.4691
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7445 - loss: 0.8517 - val_accuracy: 0.9193 - val_loss: 0.2541
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6725 - loss: 1.0839
Test loss: 1.0160973072052002
Test accuracy: 0.6819671988487244


In [ ]:
from k3im.simple_vit_3d import SimpleViT3DModel

model = SimpleViT3DModel(
    image_size=28,
    image_patch_size=7,
    frames=28,
    frame_patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=4,
    mlp_dim=32,
    channels=1,
    dim_head=64,
)

In [ ]:
model.summary()

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, 28, 28,    │       0 │ -                    │
│ (InputLayer)        │ 28, 1)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_12          │ (None, 7, 4, 7,   │       0 │ input_layer_26[0][0] │
│ (Reshape)           │ 4, 7, 4, 1)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_4         │ (None, 4, 4, 4,   │       0 │ reshape_12[0][0]     │
│ (Transpose)         │ 7, 7, 7, 1)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_13          │ (None, 4, 4, 4,   │       0 │ transpose_4[0][0]    │
│ (Reshape)           │ 343)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │     686 │ reshape_13[0][0]     │
│ (LayerNormalizatio… │ 343)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_52 (Dense)    │ (None, 4, 4, 4,   │  11,008 │ layer_normalization… │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │      64 │ dense_52[0][0]       │
│ (LayerNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_14          │ (None, 64, 32)    │       0 │ layer_normalization… │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_14 (Add)        │ (None, 64, 32)    │       0 │ reshape_14[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 64, 32)    │  33,568 │ add_14[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_14[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_15 (Add)        │ (None, 64, 32)    │       0 │ add_14[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_22       │ (None, 64, 32)    │   2,176 │ add_15[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_16 (Add)        │ (None, 64, 32)    │       0 │ add_15[0][0],        │
│                     │                   │         │ sequential_22[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 64, 32)    │  33,568 │ add_16[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_16[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_17 (Add)        │ (None, 64, 32)    │       0 │ add_16[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_23       │ (None, 64, 32)    │   2,176 │ add_17[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_18 (Add)        │ (None, 64, 32)    │       0 │ add_17[0][0],        │
│                     │                   │         │ sequential_23[0][0]

 Total params: 83,673 (326.85 KB)

 Trainable params: 83,673 (326.85 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_model(model)

Epoch 1/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step - accuracy: 0.7908 - loss: 0.6558 - val_accuracy: 0.9317 - val_loss: 0.2221
Epoch 2/2
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8924 - loss: 0.3603 - val_accuracy: 0.9317 - val_loss: 0.1682
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7327 - loss: 1.0193
Test loss: 0.923545241355896
Test accuracy: 0.7344262003898621


In [ ]:
from k3im.vit_3d import ViT3DModel

In [ ]:
model = ViT3DModel(
    image_size=28,
    image_patch_size=7,
    frames=28,
    frame_patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=4,
    mlp_dim=32,
    pool='cls',
    channels=3,
    dim_head=64,
) ############ ERRRRRRRRRRRRRRRRRRRRRRRRRRR

In [ ]:
model.summary()

Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 28, 28,    │       0 │ -                    │
│ (InputLayer)        │ 28, 3)            │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_15          │ (None, 7, 4, 7,   │       0 │ input_layer_23[0][0] │
│ (Reshape)           │ 4, 7, 4, 3)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_8         │ (None, 4, 4, 4,   │       0 │ reshape_15[0][0]     │
│ (Transpose)         │ 7, 7, 7, 3)       │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_16          │ (None, 4, 4, 4,   │       0 │ transpose_8[0][0]    │
│ (Reshape)           │ 1029)             │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │   2,058 │ reshape_16[0][0]     │
│ (LayerNormalizatio… │ 1029)             │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_54 (Dense)    │ (None, 4, 4, 4,   │  32,960 │ layer_normalization… │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 4,   │      64 │ dense_54[0][0]       │
│ (LayerNormalizatio… │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_17          │ (None, 64, 32)    │       0 │ layer_normalization… │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ class_token_positi… │ (None, 65, 32)    │   2,112 │ reshape_17[0][0]     │
│ (ClassTokenPositio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 65, 32)    │  33,568 │ class_token_positio… │
│ (MultiHeadAttentio… │                   │         │ class_token_positio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_10 (Add)        │ (None, 65, 32)    │       0 │ class_token_positio… │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_17       │ (None, 65, 32)    │   2,176 │ add_10[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_11 (Add)        │ (None, 65, 32)    │       0 │ add_10[0][0],        │
│                     │                   │         │ sequential_17[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 65, 32)    │  33,568 │ add_11[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_11[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_12 (Add)        │ (None, 65, 32)    │       0 │ add_11[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_18       │ (None, 65, 32)    │   2,176 │ add_12[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_13 (Add)        │ (None, 65, 32)    │       0 │ add_12[0][0],      

 Total params: 109,109 (426.21 KB)

 Trainable params: 109,109 (426.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_model(model)